In [1]:
import xarray as xr
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seastar as ss
import seastar.oscar
import seastar.retrieval
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
import os
from scipy import interpolate

In [2]:
#version = 'v20230119'
#version = 'v20221208_2'
#version = 'FINAL'
#flight = '20220517'
version = 'v20221104_2'
file_paths = ss.utils.readers._set_file_paths()
oscar_path = os.path.join(file_paths['oscar_path'], version, 'matlab-pre-processed','')
file_time_triplets = ss.utils.tools.find_file_triplets(oscar_path)

print('OSCAR L1 processing - version ' + version)
print('-------------------')
print('File path :', oscar_path)
print('-------------------')
for i in range(len(file_time_triplets)):
    print('File index :', i, ',', 'Aquisition time :', file_time_triplets[i][0], ',', 'File list indices :', file_time_triplets[i][1])
date = file_time_triplets[0][0].split('T')[0]
DAR_tracks = ss.utils.readers._read_DAR_config(date)
# L1a variables to be passed to L1b dataset
vars_to_keep = [
        'LatImage',
        'LonImage',
        'IncidenceAngleImage',
        'LookDirection',
        'SquintImage',
        'CentralFreq',
        ]
# Compute L1C - Sigma0 + RVL
ds_L1b = dict()
#for track in DAR_tracks.keys():
for track in ['Track_12']:
    ds_ml = dict()
    file_index = DAR_tracks[track]
    ds_L1a = ss.utils.readers.load_OSCAR_data(oscar_path, file_time_triplets[file_index][1])
    
    ds_L1a_dict_index = list(ds_L1a.keys())
    antenna_ident = ss.utils.tools.identify_antenna_location_from_filename(
                        oscar_path,
                        file_time_triplets[file_index][1])
    print('Processing file index',file_index,'...')
    
    # L1b processing - loop through beams in L1a dataset dict
    #--------------------------------------------------------
    for dict_index in ds_L1a_dict_index:
        ds_L1a[dict_index] = ss.oscar.level1.replace_dummy_values(
                                 ds_L1a[dict_index],
                                 dummy_val=int(ds_L1a[dict_index].Dummy.data))
        ds_ml[dict_index] = ss.oscar.level1.compute_multilooking_Master_Slave(
                                ds_L1a[dict_index],
                                window=7)
        ds_ml[dict_index]['Polarization'] = ss.oscar.level1.check_antenna_polarization(ds_L1a[dict_index])
        ds_ml[dict_index]['Baseline'] = ss.oscar.level1.compute_antenna_baseline(0.2)
        ds_ml[dict_index]['AntennaAzimuthImage'] =ss.oscar.level1.compute_antenna_azimuth_direction(
                                                    ds_L1a[dict_index],
                                                    antenna=antenna_ident[ds_L1a_dict_index.index(dict_index)])
        ds_ml[dict_index]['TimeLag'] = ss.oscar.level1.compute_time_lag_Master_Slave(
                                           ds_L1a[dict_index],
                                           options='from_SAR_time')
        ds_ml[dict_index][vars_to_keep] = ds_L1a[dict_index][vars_to_keep]
        ds_ml[dict_index]['RadialSurfaceVelocity'] = ss.oscar.level1.compute_radial_surface_velocity(
                                                         ds_ml[dict_index])
        
        ds_ml[dict_index].attrs['Title'] = ds_L1a[dict_index].Title
        ds_ml[dict_index]['TrackTime'] = ss.oscar.level1.track_title_to_datetime(ds_ml[dict_index].Title)
#-----------------------------------------------------------
    ds_ml = ss.oscar.level1.fill_missing_variables(ds_ml, antenna_ident)
#
# Build L1b dataset

    ds_L1b[track] = ss.oscar.level1.merge_beams(ds_ml, antenna_ident)
    del ds_ml
    
print('...Done')

Device name = LIVPCZG-10
Setting local paths...
OSCAR L1 processing - version v20221104_2
-------------------
File path : D:\data\SEASTAR\SEASTARex\Data\Metasensing\OSCAR\v20221104_2\matlab-pre-processed\
-------------------
File index : 0 , Aquisition time : 20220522T052218 , File list indices : [0, 1, 2]
File index : 1 , Aquisition time : 20220522T053942 , File list indices : [3, 4, 5]
File index : 2 , Aquisition time : 20220522T054805 , File list indices : [6, 7, 8]
File index : 3 , Aquisition time : 20220522T055458 , File list indices : [9, 10, 11]
File index : 4 , Aquisition time : 20220522T062018 , File list indices : [12, 13, 14]
File index : 5 , Aquisition time : 20220522T062620 , File list indices : [15, 16, 17]
File index : 6 , Aquisition time : 20220522T063239 , File list indices : [18, 19, 20]
File index : 7 , Aquisition time : 20220522T063906 , File list indices : [21, 22, 23]
File index : 8 , Aquisition time : 20220522T064459 , File list indices : [24, 25, 26]
File index 

In [12]:
folder = 'restricted'
data_type = 'METEOFRANCE_AROME'
AROME_data_folder = os.path.join(file_paths['aux_path'], folder, data_type,'')

AROME_dict = dict()
AROME_file_list = os.listdir(AROME_data_folder)
for file in AROME_file_list:
    AROME_dict[AROME_file_list.index(file)] =\
    xr.open_dataset(AROME_data_folder + file)

AROME_winds = xr.concat(list(AROME_dict.values()),
                        dim='time',
                       join='outer')
del AROME_dict
ds_L2 = dict()
for track in ds_L1b.keys():
    print('Processing ',track,'...')
    # Generating geophysical data fields for GMF doppler computation
    u10 = AROME_winds.u10m.interp(time=ds_L1b[track].sel(Antenna='Mid').TrackTime,method='nearest')\
        .interp(latitude=ds_L1b[track].latitude.median(), method='nearest')\
        .interp(longitude=ds_L1b[track].longitude.median(), method='nearest')
    v10 = AROME_winds.v10m.interp(time=ds_L1b[track].sel(Antenna='Mid').TrackTime,method='nearest')\
        .interp(latitude=ds_L1b[track].latitude.median(), method='nearest')\
        .interp(longitude=ds_L1b[track].longitude.median(), method='nearest')
    wind_speed, wind_direction = ss.utils.tools.windUV2SpeedDir(u10, v10)
    aux = ss.oscar.level1.init_auxiliary(ds_L1b[track], wind_speed.data, wind_direction.data)
    
    # Compute RSC
    rsc = [compute_radial_surface_current(ds_L1b[track].sel(Antenna=a), aux, gmf='mouche12') for a in antenna_ident]
    ds_L1b[track]['RadialSurfaceCurrent'] = xr.concat(rsc, 'Antenna',join='outer')
    #ds_L1b[track] = ss.oscar.level1.compute_radial_surface_current(ds_L1b[track], aux, gmf='mouche12')
    #ds_L1b[track]  = compute_radial_surface_current(ds_L1b[track] , aux, gmf='mouche12')
    
    
    #L2 Processing
    # Initialise l2 dataset
    ds_L2[track] = ss.oscar.level1.init_level2(ds_L1b[track])
    ds_L2[track].merge(aux)
    # Compute current magnitude and direction
    ds_L2[track] = seastar.retrieval.level2.compute_current_magnitude_and_direction(ds_L1b[track], ds_L2[track])
    # Compute current vectors
    ds_L2[track]['CurrentVectorUComponent'], ds_L2[track]['CurrentVectorVComponent'] =\
    ss.utils.tools.currentVelDir2UV(ds_L2[track]['CurrentMagnitude'], ds_L2[track]['CurrentDirection'])
    
    ds_L2[track]['TrackTime'] = ds_L1b[track].TrackTime.sel(Antenna='Mid').data
    ds_L2[track][vars_to_keep] = ds_L1b[track][vars_to_keep]
print('...done.')

Processing  Track_12 ...


C:\Users\davidmccann\anaconda3\envs\seastar\lib\site-packages\xarray\core\computation.py:771: RuntimeWarning: divide by zero encountered in true_divide
  result_data = func(*input_data)


...done.


In [ ]:
ds_L2[track].CurrentMagnitude.plot(x='longitude',y='latitude',
                                  vmin=0,vmax=1)

In [4]:
def compute_radial_surface_current(level1, aux, gmf='mouche12'):

    dswasv = seastar.gmfs.doppler.compute_wasv(level1,
                                               aux,
                                               gmf
                                               )
    radial_surface_current = xr.DataArray(data=level1.RadialSurfaceVelocity - dswasv,
                                          coords=level1.RadialSurfaceVelocity.coords,
                                          dims=level1.RadialSurfaceVelocity.dims)
    radial_surface_current.attrs['long_name'] =\
        'Radial Surface Current'
    radial_surface_current.attrs['description'] =\
        'Radial Surface Current (RSC) along antenna beam direction, corrected'\
        'for Wind Artifact Surface Velocity (WASV)'
    radial_surface_current.attrs['units'] = 'm/s'                         

#    warnings.warn(
#        "WARNING: Applying direction convention correction on the Aft beam,"
#        "Be aware this may be an obsolete correction in the future and will"
#        "lead to an error in retrieved current direction."
#    )
#    rsv_list[list(level1.Antenna.data).index('Aft')] = \
#        -level1.RadialSurfaceVelocity.sel(Antenna='Aft')\
#        - dswasv.sel(Antenna='Aft')
    
    return radial_surface_current

In [7]:
rsc = [compute_radial_surface_current(ds_L1b[track].sel(Antenna=a), aux, gmf='mouche12') for a in antenna_ident]
ds_L1b[track]['RadialSurfaceCurrent'] = xr.concat(rsc, 'Antenna',join='outer')

C:\Users\davidmccann\anaconda3\envs\seastar\lib\site-packages\xarray\core\computation.py:771: RuntimeWarning: divide by zero encountered in true_divide
  result_data = func(*input_data)


In [53]:
my_dict = {'active_mask': np.array([0, 0, 0, 0]),'cost': 15.580097200687781,'fun': np.array([ 2.23880093,-1.18788098,-0.91092943,0.92854008,-2.89013191,
                                                                                             0,0,-3.83302257]),
           'grad': np.array([-3.70908189,-5.42794954,543.90114085,749.69315317]),
           'jac': np.array([[1.00893814,-2.72094499,-1.00893834,2.72094455],
                            [4.01131692,-1.47655665,-4.01131682,1.47655688],
                            [3.85659753,-1.70178855,-3.85659761,1.70178864],
                            [2.48795051,-2.84368792,-2.48795079,2.84368767],
                            [-0,0,-222.2142992,-222.21429628],
                            [-0,0,0,-0],
                            [-0,0,0,-0],
                            [-0,0,26.62039557,-26.62039557]]),
           'message': '`xtol` termination condition is satisfied.',
           'nfev': 22,
           'njev': 20,
           'optimality': 3244.0962317041167,
           'status': 3,
           'success': True,
           'x': np.array([-5.13697711,7.08607154,0.55802105,-0.67276796])}

In [56]:
my_dict = {'active_mask': np.array([0, 0, 0, 0])}

In [35]:
np.array([ 2.23880093, -1.18788098, -0.91092943, 0.92854008, -2.89013191,0,0,-3.83302257])

array([ 2.23880093, -1.18788098, -0.91092943,  0.92854008, -2.89013191,
        0.        ,  0.        , -3.83302257])

In [29]:
my_dict = {'active_mask': np.array([0, 0, 0, 0]),'cost': 15.580097200687781}
my_dict.keys()

dict_keys(['active_mask', 'cost'])

In [52]:
my_ds=xr.Dataset()
for var in my_dict.keys():
    my_ds[var]=my_dict[var]
    
my_ds

<xarray.Dataset>
Dimensions:      (active_mask: 4, fun: 8, grad: 4, x: 4)
Coordinates:
  * active_mask  (active_mask) int32 0 0 0 0
  * fun          (fun) float64 2.239 -1.188 -0.9109 0.9285 -2.89 0.0 0.0 -3.833
  * grad         (grad) float64 -3.709 -5.428 543.9 749.7
  * x            (x) float64 -5.137 7.086 0.558 -0.6728
Data variables:
    cost         float64 15.58
    message      <U42 '`xtol` termination condition is satisfied.'
    nfev         int32 22
    njev         int32 20
    optimality   float64 3.244e+03
    status       int32 3
    success      bool True

In [69]:
a = {1:2, 3:4}
print(a)
del a[1]
print(a)

{1: 2, 3: 4}
{3: 4}
